<a href="https://colab.research.google.com/github/taaha3244/llama-index/blob/main/llamaindex_kg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install llama-index llama-index-graph-stores-neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 24.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
     ━━━━

In [1]:
!unzip 'kg'

Archive:  kg.zip
  inflating: kg/2023 Qualified Allocation Plan.pdf  


In [2]:
from google.colab import userdata

In [3]:
import os

os.environ["OPENAI_API_KEY"] =userdata.get('openai_api_key')

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [6]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="/content/kg")
documents = reader.load_data()

In [7]:
documents

[Document(id_='31862dd3-8174-466f-b458-c337b67a22e3', embedding=None, metadata={'page_label': '1', 'file_name': '2023 Qualified Allocation Plan.pdf', 'file_path': '/content/kg/2023 Qualified Allocation Plan.pdf', 'file_type': 'application/pdf', 'file_size': 1236820, 'creation_date': '2024-06-05', 'last_modified_date': '2024-06-06'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Department of Housing and Community Development  \n1800 Martin Luther King Jr. Avenue SE | Washington, DC 20020  2023\nLOW INCOME HOUSING TAX CR EDIT \nQ ualified Allocation  Plan \nAugust 2023 ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='bd1e6d11-db30-4e98-911

In [10]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

# Note: used to be `Neo4jPGStore`
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password=userdata.get('neo4j_password'),
    url=userdata.get('neo4j_url')
)

In [14]:
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0)
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/59 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1493, in request
    return await self._request(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1546, in _request
    return await self._retry_request(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1615, in _retry_request
    return await self._request(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1546, in _request
    return await self._retry_request(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1615, in _retry_request
    return await self._request(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1546, in _request
    return await self._retry_request(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1615, in _retry_request
    return await self._request(
  File "/usr/local

In [16]:
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

nodes = retriever.retrieve("What Financial Criteria are there for solar realted projects?")

for node in nodes:
    print(node.text)

Threshold Eligibility Requirements -> HAS -> Financial Criteria
Solar ITC -> USED_FOR -> Tax Benefits for Solar Panel Installation
Threshold Eligibility Requirements -> HAS -> Project Criteria
Project Criteria -> HAS -> Project Eligibility and Certifications
Solar ITC -> IS_A -> Solar Investment Tax Credit


In [20]:
retriever = index.as_retriever(
    include_text=True,  # include source text in returned nodes, default True
)

nodes = retriever.retrieve("What Financial Criteria are there for projects?")

for node in nodes:
    print(node.text)

Here are some facts extracted from the provided text:

Threshold Eligibility Requirements -> HAS -> Financial Criteria
Threshold Eligibility Requirements -> HAS -> Project Criteria
Project Criteria -> HAS -> Project Eligibility and Certifications

DHCD 2023 Qualified Allocation Plan, Page 19 of 59  
The Threshold Eligibility Requirements pursuant to this 2023 QAP are more fully described below:  
 
  
  
  
 
THRESHOLD  ELIGIBILITY  REQU IREMENT S Applicable?  
Project Criteria    
 Project  Eligibility  and Certifications    
 Permanent  Supportive  Housin g  NA 
 Site Control    
 Entitlements and Development Review    
Financial  Criteria    
 Development  Budget and Operat ing Proforma    
 Acquisition Cost Reasonableness    
 Financin g Letters  and Terms    
 Financial  Information  for Projects  already in  operation  NA  
Applicant  Criteria    
 Development  Team  Thresholds    
Reports  and Plans    
 Appraisal    
 Market  Study and Market Demand Analysis  

In [24]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("""Can you help me find 5 financing options for the following project?List each option with benefits and criteria . Please guide me in detail
Thee owner of Marbury Plaza wants to finance a comprehensive, deep energy retrofit of the property, including a 500kW solar project on the roof. The retrofit measures include:

Envelope Improvements: Insulation, windows, air tightness, etc.

Electric Lighting: Fixture upgrades, etc.

Efficient Equipment

Heating, Cooling & Ventilation

Total Cost:

Retrofit Measures: $50,000,000

Rooftop Solar Project: $1,500,000

Total: $51,500,000""")

print(str(response))

1. **Low-Income Housing Tax Credits (LIHTCs):**
   - **Benefits:** LIHTCs provide a dollar-for-dollar reduction in federal income tax liability for affordable housing developers.
   - **Criteria:** Projects must involve new construction, acquisition, or rehabilitation and meet the basic eligibility requirements of the tax credit program.

2. **Energy Efficiency and Renewable Energy Grants:**
   - **Benefits:** Grants can provide funding for energy efficiency improvements and renewable energy projects.
   - **Criteria:** Eligibility criteria may vary based on the specific grant program, but typically focus on promoting energy efficiency and sustainability.

3. **PACE Financing (Property Assessed Clean Energy):**
   - **Benefits:** PACE financing allows property owners to finance energy efficiency and renewable energy projects through a special assessment on their property tax bill.
   - **Criteria:** Property owners must meet certain requirements set by the PACE program, such as having 

#Loading from an existing Graph

In [5]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password=userdata.get('neo4j_password'),
    url=userdata.get('neo4j_url')
)

index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
)

In [11]:
retriever = index.as_retriever(
    include_text=True,
# include source text in returned nodes, default True
)

nodes = retriever.retrieve(""""Please read the documentation and extract the information I need:


1. Project summary
2. list of Criteria, by category, and in each category there might be multiple criteria, list them by criteria name, condition, criteria term. Possible category could be:
 - Project Criteria
 - Financial Criteria
 - Applicant Criteria
 - Compliance Criteria
3. Is there any project Restrictions
4. Project EVALUATION CRITERIA List
5 AVAILABLE FUNDING SOURCES, for each funding type, find out funding name, Assistance Type, Terms, Restrictions
6. Reports and Plans required for the project"

The project is as follows :

Thee owner of Marbury Plaza wants to finance a comprehensive, deep energy retrofit of the property, including a 500kW solar project on the roof. The retrofit measures include:

Envelope Improvements: Insulation, windows, air tightness, etc.

Electric Lighting: Fixture upgrades, etc.

Efficient Equipment

Heating, Cooling & Ventilation

Total Cost:

Retrofit Measures: $50,000,000

Rooftop Solar Project: $1,500,000

Total: $51,500,000""")

for node in nodes:
    print(node.text)

Here are some facts extracted from the provided text:

Applicable Percentage -> IS_A -> Project
Project -> USED_FOR -> Rent Control law
Project -> USED_FOR -> affordability restrictions
Project -> USED_FOR -> Net New Units
Project -> USED_FOR -> DCHFA’s Mortgage Revenue Bond Program
Project -> USED_FOR -> IRC §42
Project -> USED_FOR -> Threshold Eligibility Requirements
Project -> USED_FOR -> 4% Tax Credits
Project -> USED_FOR -> 9% Tax Credits

DHCD 2023  Qualified Allocation Plan, Page 46 of 59  Definitions  
 
The following  capitalized  terms  shall have  the meanings  below unless  context  clearly  requires  
a different meaning.  
Accelerator Program: A  formal initiative provid ing career advancement guidance to real 
estate developers of color  who are focused on affordable housing and inclusive community 
building. K ey characteristics of accelerator programs include professional training  and 
fellowships , mentorship, and capital fundi ng. 
Affiliate: A corporation, partner

In [14]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query(""""Generate a detailed list of Criteria, by category, and in each category there might be multiple criteria, list them by criteria name, condition, criteria term.




The project is as follows :

Thee owner of Marbury Plaza wants to finance a comprehensive, deep energy retrofit of the property, including a 500kW solar project on the roof. The retrofit measures include:

Envelope Improvements: Insulation, windows, air tightness, etc.

Electric Lighting: Fixture upgrades, etc.

Efficient Equipment

Heating, Cooling & Ventilation

Total Cost:

Retrofit Measures: $50,000,000

Rooftop Solar Project: $1,500,000

Total: $51,500,000""")

print(str(response))

Development Budget and Operating Proforma  
- Criteria Name: Development Budget and Operating Proforma  
- Condition: Applicable  
- Criteria Term: Must provide a detailed budget and operating proforma for the project  

Acquisition Cost Reasonableness  
- Criteria Name: Acquisition Cost Reasonableness  
- Condition: Applicable  
- Criteria Term: Must demonstrate that the acquisition cost is reasonable for the project  

Financing Letters and Terms  
- Criteria Name: Financing Letters and Terms  
- Condition: Applicable  
- Criteria Term: Must provide financing letters and terms for the project  

Financial Information for Projects already in operation  
- Criteria Name: Financial Information for Projects already in operation  
- Condition: Not Applicable  
- Criteria Term: Not required for this project  

Green Building Design and Construction  
- Criteria Name: Green Building Design and Construction  
- Condition: Applicable  
- Criteria Term: Must incorporate green building design a

In [15]:
print(str(response))

Development Budget and Operating Proforma  
- Criteria Name: Development Budget and Operating Proforma  
- Condition: Applicable  
- Criteria Term: Must provide a detailed budget and operating proforma for the project  

Acquisition Cost Reasonableness  
- Criteria Name: Acquisition Cost Reasonableness  
- Condition: Applicable  
- Criteria Term: Must demonstrate that the acquisition cost is reasonable for the project  

Financing Letters and Terms  
- Criteria Name: Financing Letters and Terms  
- Condition: Applicable  
- Criteria Term: Must provide financing letters and terms for the project  

Financial Information for Projects already in operation  
- Criteria Name: Financial Information for Projects already in operation  
- Condition: Not Applicable  
- Criteria Term: Not required for this project  

Green Building Design and Construction  
- Criteria Name: Green Building Design and Construction  
- Condition: Applicable  
- Criteria Term: Must incorporate green building design a